In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('intersection_dataset_modified.csv')
df.head(-5)

,id,title,genres,Release Year,Origin/Ethnicity,Director,Cast,PlotSummary
0,31186339,the hunger games,"['Action/Adventure', 'Science Fiction', 'Actio...",2012,American,Gary Ross,"Jennifer Lawrence, Josh Hutcherson, Liam Hemsw...",Katniss Everdeen volunteers to take her place ...
1,20663735,narasimham,"['Musical', 'Action', 'Drama', 'Bollywood']",2000,Malayalam,Shaji Kailas,"Mohanlal, Aiswarya","Induchoodan, the only son of Justice Marancher..."
2,2231378,the lemon drop kid,"['Screwball comedy', 'Comedy']",1934,American,Marshall Neilan,"Lee Tracy, Helen Mack",Con artist and racetrack tout Wally Brooks han...
3,2231378,the lemon drop kid,"['Screwball comedy', 'Comedy']",1951,American,Sidney Lanfield,"Bob Hope, Marilyn Maxwell, William Frawley",The Lemon Drop Kid (Bob Hope) is illegally tou...
4,595909,a cry in the dark,"['Crime Fiction', 'Drama', 'Docudrama', 'World...",1988,American,Fred Schepisi,"Meryl Streep, Sam Neill",Meryl Streep stars as seventh-day Adventist Ch...
...,...,...,...,...,...,...,...,...
24980,27978238,westbound,['Western'],1959,American,Budd Boetticher,"Randolph Scott, Virginia Mayo, Karen Steele",Captain John Hayes is asked to take charge of ...
24981,871575,avalon,"['Thriller', 'Science Fiction', 'Japanese Movi...",1990,American,Barry Levinson,"Aidan Quinn, Armin Mueller-Stahl, Joan Plowrig...",Russian Jewish immigrant Sam Krichinsky's fami...
24982,871575,avalon,"['Thriller', 'Science Fiction', 'Japanese Movi...",2001,Japanese,Mamoru Oshii,NaN,Avalon is a military-themed virtual reality sh...
24983,29046736,i don't want to be born,['Horror'],1975,British,Peter Sasdy,"Joan Collins, Ralph Bates, Eileen Atkins",Lucy (Collins) is working as a dancer in a sle...


In [ ]:
pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load dataset
df = pd.read_csv("intersection_dataset_modified.csv")
plot_summaries = df["PlotSummary"].tolist()
titles = df["title"].tolist()

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight model
embeddings = model.encode(plot_summaries, convert_to_numpy=True, normalize_embeddings=True)

# Step 2: Create FAISS index for cosine similarity
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # Inner Product for cosine similarity
faiss.normalize_L2(embeddings)  # Normalize embeddings
index.add(embeddings)  # Add embeddings to index

# Save FAISS index
faiss.write_index(index, "movie_plots_index.faiss")

In [ ]:
res = faiss.StandardGpuResources()
gpu_index = faiss.index_cpu_to_gpu(res, 0, index)

# Save titles mapping for retrieval
df.to_csv("titles_mapping.csv", index=False)

# Function to match user query
def find_best_match(user_query):
    query_embedding = model.encode([user_query], convert_to_numpy=True, normalize_embeddings=True)
    distances, indices = gpu_index.search(query_embedding, k=1)  # Retrieve top match
    best_index = indices[0][0]
    return titles[best_index], distances[0][0]

# Example usage
user_query = "A story about a young wizard fighting evil forces."
best_title, score = find_best_match(user_query)
print(f"Best match: {best_title} (score: {score:.4f})")


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Best match: sword stained with royal blood (score: 0.4116)


In [ ]:
def find_top_k_matches(user_query, k=3):
    # Generate embedding for the user query
    query_embedding = model.encode([user_query], convert_to_numpy=True, normalize_embeddings=True)

    # Search for the top-k matches in the FAISS index
    distances, indices = gpu_index.search(query_embedding, k=k)

    # Retrieve titles and scores for the top-k matches
    results = []
    for i in range(k):
        match_index = indices[0][i]
        match_score = distances[0][i]
        match_title = titles[match_index]
        results.append((match_title, match_score))

    return results

# Example usage
user_query = "A hacker discovers a simulated reality controlled by machines."
top_k_results = find_top_k_matches(user_query, k=3)

# Print the results
print("Top matches:")
for rank, (title, score) in enumerate(top_k_results, start=1):
    print(f"{rank}. {title} (score: {score:.4f})")


Top matches:
1. gamer (score: 0.4797)
2. the matrix (score: 0.4559)
3. weird science (score: 0.4458)
